In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tflearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 107 kB 15.1 MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=7f94a0b0b82739ccadd626db2c8c2a46cef4b9ec69a4dfaa6299d91210e51d8f
  Stored in directory: /root/.cache/pip/wheels/65/9b/15/cb1e6b279c14ed897530d15cfd7da8e3df8a947e593f5cfe59
Successfully built tflearn


In [ ]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
import numpy as np
import pandas as pd
import tflearn
import tensorflow as tf
import json
import pickle
import random
nltk.download("punkt")

Instructions for updating:
non-resource variables are not supported in the long term
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
#Loading intents.json
with open('/content/drive/MyDrive/Smstr 5 (AI 4 StartUps)/dataset_chatbot.json') as intents:
  data = json.load(intents)

stemmer = LancasterStemmer()

# getting informations from intents.json--
words = []
labels = []
x_docs = []
y_docs = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        x_docs.append(wrds)
        y_docs.append(intent['tag'])

        if intent['tag'] not in labels:
            labels.append(intent['tag'])

In [ ]:
# # Konversi data json ke dalam dataframe
# pattern1 = pattern
# labels1 = labels
# data = pd.DataFrame({"patterns":pattern1, "tags":labels1})
# data

In [ ]:
# Stemming the words and removing duplicate elements.
words = [stemmer.stem(w.lower()) for w in words if w not in "?"]
words = sorted(list(set(words)))
labels = sorted(labels)

In [ ]:
training = []
output = []
out_empty = [0 for _ in range(len(labels))]

# One hot encoding, Converting the words to numerals
for x, doc in enumerate(x_docs):
    bag = []
    wrds = [stemmer.stem(w) for w in doc]
    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)


    output_row = out_empty[:]
    output_row[labels.index(y_docs[x])] = 1

    training.append(bag)
    output.append(output_row)


training = np.array(training)
output = np.array(output)

In [ ]:
net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, len(output[0]), activation='softmax')
net = tflearn.regression(net)

model = tflearn.DNN(net)
model.fit(training, output, n_epoch=500, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 6499  | total loss: 0.01094 | time: 0.030s
| Adam | epoch: 500 | loss: 0.01094 - acc: 1.0000 -- iter: 096/104
Training Step: 6500  | total loss: 0.01031 | time: 0.033s
| Adam | epoch: 500 | loss: 0.01031 - acc: 1.0000 -- iter: 104/104
--


In [ ]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]
    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for s_word in s_words:
        for i, w in enumerate(words):
            if w == s_word:
                bag[i] = 1

    return np.array(bag)

In [ ]:
def chat():

    while True:
        inp = input("\n\nYou: ")
        if inp.lower() == 'quit':
            break

    #Porbability of correct response 
        results = model.predict([bag_of_words(inp, words)])

    # Picking the greatest number from probability
        results_index = np.argmax(results)

        tag = labels[results_index]

        for tg in data['intents']:

            if tg['tag'] == tag:
                responses = tg['responses']
                print("Bot:\t" + random.choice(responses))
                

In [ ]:
chat()



You: hello
Bot:	Dadahh!


You: candi
Bot:	Ada Candi Borobudur, Candi Prambanan, Candi Ratu Boko, Candi Sewu


You: jalan
Bot:	kamu bisa membeli Bakpia kencana, bakpia pathuk, yangko, walang goreng, bakpia 45, bakpia kurnia sari, bakpiaku, bakpia tugu


You: makan
Bot:	Dengan senang hati:)


KeyboardInterrupt: ignored